# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BenchmarkTools

Threads.nthreads() == 1 && 
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`");
println("Number of threads: $(Threads.nthreads())")

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


Number of threads: 4


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [2]:
function LoopFields(c, model)
    model === :On && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    )
    model === :PSUn && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
    model === :Potts && return vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
end

LoopSpectrum(c, model, Δmax) = Spectrum(LoopFields(c, model), Δmax, interchiral=true);

function solve(;indices, signature, parity, even_spin=(), benchmark=false)
    ext_fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(ext_fields..., Δmax=Δmax)
    SOn = LoopSpectrum(c, :On, Δmax)
    Seven = Spectrum([V for V in SOn.fields if spin(V) % 2 == 0], Δmax, interchiral=true)

    specs = (; (chan =>
        if chan in even_spin
            Seven
        else
            SOn
        end
                for chan in (:s, :t, :u)
    )...)

    chan_spectra = ChannelSpectra(
        co, specs, signature;
        interchiral=true, Δmax=Δmax, parity=1
    )

    sys = BootstrapSystem(chan_spectra)
    evaluate_blocks!(sys)
    compute_linear_system!(sys)
    solve!(sys)
    if benchmark
        println("precomputation of blocks coefficients")
        @btime ChannelSpectra(
            $co, $SOn, $signature;
            interchiral=true, Δmax=Δmax, parity=1,
            exclude=(u=[Field(c, r=2, s=0)],)
        )
        println("time to compute all positions' cache")
        @btime BootstrapSystem($chan_spectra)
        println("time to evaluate blocks")
        @btime evaluate_blocks!($sys)
        println("time for the linear solver")
        @btime solve!($sys)
    end
    return sys
end

solve (generic function with 1 method)

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [ ]:
LeftRight((1, 2))


TypeError: TypeError: in typeassert, expected @NamedTuple{left::T, right::T} where T, got a value of type Tuple{Int64, Int64}

In [4]:
Δmax, prec = 30, 25
setprecision(BigFloat, prec, base=10);
c = CC(β=1 / (big"0.8" + big"0.1" * im))
ind = ((1 // 2, 0), (1 // 2, 0), (1 // 2, 0), (1 // 2, 0))
sig = (s=0, t=1, u=1)
sol = solve(indices=ind, signature=sig, parity=1, even_spin=(:s,), benchmark=false)
# println("total time to setup and solve the crossing equations:")
# @btime solve(indices=ind, signature=sig, parity=1, even_spin=(:s,))
println(sol.structure_constants)

MethodError: MethodError: no method matching (@NamedTuple{left::T, right::T} where T)(::Tuple{ConformalDimension{Complex{BigFloat}}, ConformalDimension{Complex{BigFloat}}})
The type `@NamedTuple{left::T, right::T} where T` exists, but no method is defined for this combination of argument types when trying to construct it.

In [5]:
@btime BootstrapVirasoro.compute_reference!(sol)

UndefVarError: UndefVarError: `sol` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
total_dimension(Field(c, r=6, s=0))
sol.spectra.s.Δmax

MethodError: MethodError: no method matching Union{Int64, Rational}(::Bool)
The type `Union{Int64, Rational}` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  (::Type{T})(::T) where T<:Number
   @ Core boot.jl:900
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number}
   @ Base char.jl:50
  (::Type{T})(!Matched::Complex) where T<:Real
   @ Base complex.jl:44
  ...


Benchmarks:
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [7]:
ind = ((1, 0), (1, 0), (1//2, 0), (1//2, 0))
sig = (s=0, t=3//2, u=3//2)
println(solve(indices=ind, signature=sig, parity=1, even_spin=(:s,)).structure_constants)

MethodError: MethodError: no method matching Union{Int64, Rational}(::Bool)
The type `Union{Int64, Rational}` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  (::Type{T})(::T) where T<:Number
   @ Core boot.jl:900
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number}
   @ Base char.jl:50
  (::Type{T})(!Matched::Complex) where T<:Real
   @ Base complex.jl:44
  ...


In [8]:
sig = (s=1, t=1//2, u=3//2)
println(solve(indices=ind, signature=sig, parity=1).structure_constants)

MethodError: MethodError: no method matching Union{Int64, Rational}(::Bool)
The type `Union{Int64, Rational}` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  (::Type{T})(::T) where T<:Number
   @ Core boot.jl:900
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number}
   @ Base char.jl:50
  (::Type{T})(!Matched::Complex) where T<:Real
   @ Base complex.jl:44
  ...


In [9]:
sig = (s=1, t=3//2, u=1//2)
println(solve(indices=ind, signature=sig, parity=1).structure_constants)

MethodError: MethodError: no method matching Union{Int64, Rational}(::Bool)
The type `Union{Int64, Rational}` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  (::Type{T})(::T) where T<:Number
   @ Core boot.jl:900
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number}
   @ Base char.jl:50
  (::Type{T})(!Matched::Complex) where T<:Real
   @ Base complex.jl:44
  ...


# Torus one point functions

In [10]:
V1 = Field(c, r=1, s=0)
co = Correlation(V1, Δmax=Δmax)
SOn = LoopSpectrum(c, :On, Δmax)
signature = (s=1//2, t=0, u=1//2)
schan = ChannelSpectrum(co, co.Nmax, :s)
chan_spectra = ChannelSpectra(
    co, SOn, signature;
    interchiral=true, Δmax=Δmax, parity=1
)
sys = BootstrapSystem(chan_spectra)
evaluate_blocks!(sys)
compute_linear_system!(sys)
solve!(sys)
println(sys.structure_constants)

MethodError: MethodError: no method matching Union{Int64, Rational}(::Bool)
The type `Union{Int64, Rational}` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  (::Type{T})(::T) where T<:Number
   @ Core boot.jl:900
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number}
   @ Base char.jl:50
  (::Type{T})(!Matched::Complex) where T<:Real
   @ Base complex.jl:44
  ...


In [11]:
Δmax, prec = 30, 25
setprecision(BigFloat, prec, base=10);
c = CC(β=1 / (big"0.8" + big"0.1" * im))
DG = DoubleGamma(c.β)
@btime DG(big"0.4"+big"0.1"*im)

UndefVarError: UndefVarError: `DoubleGamma` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing BarnesDoubleGamma in the current active module Main